<a href="https://colab.research.google.com/github/appersaravanan/final_project/blob/master/God_grace1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K
from keras.applications import VGG16
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import  MaxPooling2D,UpSampling2D,Input,BatchNormalization,Conv2D,Activation,MaxPool2D,Flatten,Dense
from tensorflow.keras.models import Model,Sequential,load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.preprocessing import image as image_utils
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.applications import VGG16
import cv2,os,tempfile
import requests
from keras.preprocessing import image
from scipy.io import loadmat
import tensorflow as tf
import pandas as pd
import cv2
import matplotlib.pyplot as plt

In [2]:
img_width, img_height = 224, 224
nb_train_samples = 8144
nb_validation_samples = 8041
epochs = 10
batch_size = 32
n_classes = 196

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content
!mkdir final_project
%cd ./final_project
!mkdir data
%cd data

Mounted at /content/drive
/content
/content/final_project
/content/final_project/data


In [4]:

!rm -f car_devkit.tgz
!wget https://ai.stanford.edu/~jkrause/cars/car_devkit.tgz
!tar xvzf car_devkit.tgz

--2020-12-04 18:34:17--  https://ai.stanford.edu/~jkrause/cars/car_devkit.tgz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 330960 (323K) [application/x-gzip]
Saving to: ‘car_devkit.tgz’

car_devkit.tgz      100%[===================>] 323.20K   350KB/s    in 0.9s    

2020-12-04 18:34:19 (350 KB/s) - ‘car_devkit.tgz’ saved [330960/330960]

devkit/
devkit/cars_meta.mat
devkit/cars_train_annos.mat
devkit/cars_test_annos.mat
devkit/README.txt
devkit/train_perfect_preds.txt
devkit/eval_train.m


In [ ]:
!wget http://imagenet.stanford.edu/internal/car196/cars_train.tgz

!tar xvzf cars_train.tgz
!rm -f cars_train.tgz


In [6]:
%cd /content/final_project/data

/content/final_project/data


In [ ]:
!wget http://imagenet.stanford.edu/internal/car196/cars_test.tgz
!tar xvzf cars_test.tgz
!rm -f cars_test.tgz

In [ ]:
%cd /content/final_project/data/cars_train/
annotsa = loadmat('/content/final_project/data/devkit/cars_meta.mat')
i=1
classes_name=[]
for name in annotsa['class_names'][0]:
  dir_name=str(i)+'_'+str(name[0]).replace(' ','_')
  #os.makedirs(dir_name)
  classes_name.append(dir_name)
  i=i+1
classes_name


In [ ]:
%cd /content/final_project/data/cars_test/
annotsa = loadmat('/content/final_project/data/devkit/cars_meta.mat')
i=1
classes_name=[]
for name in annotsa['class_names'][0]:
  dir_name=str(i)+'_'+str(name[0]).replace(' ','_')
  os.makedirs(dir_name)
  classes_name.append(dir_name)
  i=i+1
classes_name

In [73]:
import shutil
from shutil import copyfile
path_base='/content/final_project/data'
train_data_dir = path_base + '/cars_train'
validation_data_dir = path_base + '/cars_test'


annos = loadmat('/content/final_project/data/devkit/cars_train_annos.mat')
data = [[row.flat[0] for row in line] for line in annos['annotations'][0]]
columns = ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'class', 'fname']
df_train = pd.DataFrame(data, columns=columns)
gp=df_train.groupby('class')
files=""
for index,classes in gp:
    # print (classes_name[index-1])
    files=classes['fname']
    i=0
    for name,file in files.items():
        # print(file)
        copyfile('{}/{}'.format(train_data_dir,file),'{}/{}/{}'.format(train_data_dir,classes_name[index-1],file))
        if i<25:
          copyfile('{}/{}'.format(validation_data_dir,file),'{}/{}/{}'.format(validation_data_dir,classes_name[index-1],file))
        i=i+1


In [74]:


train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    #shear_range=0.2,
    zoom_range=0.2,
    #fill_mode = 'constant',
    #cval = 1,
    rotation_range = 5,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    classes=classes_name)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    classes=classes_name)

Found 8144 images belonging to 196 classes.
Found 4899 images belonging to 196 classes.


In [76]:

vgg16_model = VGG16(include_top=True,weights="imagenet")

model = Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)
for layer in model.layers:
    layer.trainable = False
model.add(Dense(units=196, activation='softmax'))

In [77]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc', 'mse'])

In [15]:
early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, min_delta=1e-4)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_delta=1e-4)
callbacks_list = [early_stop, reduce_lr]

In [ ]:
model.summary()
len(train_generator)

In [ ]:
model_history = model.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator
    #validation_steps=nb_validation_samples // batch_size
    #callbacks=callbacks_list
    )

Epoch 1/100
255/255 [==============================] - 197s 774ms/step - loss: 4.1029 - acc: 0.1805 - mse: 0.0047 - val_loss: 8.2306 - val_acc: 0.0045 - val_mse: 0.0057
Epoch 2/100
255/255 [==============================] - 195s 765ms/step - loss: 4.0386 - acc: 0.1928 - mse: 0.0047 - val_loss: 8.3681 - val_acc: 0.0061 - val_mse: 0.0057
Epoch 3/100
255/255 [==============================] - ETA: 0s - loss: 3.9224 - acc: 0.2015 - mse: 0.0047

In [ ]:
predictions = model.predict(x=validation_generator, verbose=0)
np.round(predictions)